In [1]:
import sys
import os
import io

# Add the parent directory to the sys.path
sys.path.append(os.path.join(os.getcwd(), '..'))

In [2]:
import glob
import csv

import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import kurtosis, skew, rankdata
from scipy.signal import argrelextrema
from scipy.stats.kde import gaussian_kde

from rcv_learning.rcv_distribution import *
from rcv_learning.rcv_dimensionality import *

C:\Users\Eric Simpson\AppData\Local\Temp\ipykernel_16372\4019494009.py:10: DeprecationWarning: Please use `gaussian_kde` from the `scipy.stats` namespace, the `scipy.stats.kde` namespace is deprecated.
  from scipy.stats.kde import gaussian_kde


In [3]:
def compute_ecdf(data):
    """Compute the empirical cumulative distribution function (ECDF) of data."""
    sorted_data = np.sort(data)
    n = len(data)
    return sorted_data, np.arange(1, n+1) / n

def dip_statistic(data):
    """Compute Hartigan's dip statistic for data."""
    data = np.sort(data)
    n = len(data)
    
    # Compute the empirical CDF
    x, ecdf = compute_ecdf(data)
    
    # Compute the greatest difference on the left and right side for each data point
    U = np.arange(1, n+1) / n
    D = U - np.arange(0, n) / n
    
    # Calculate the difference between the empirical distribution function and the 
    # unimodal distribution function that minimizes that maximum difference.
    low_dip = (ecdf - D).clip(min=0)
    up_dip = (U - ecdf).clip(min=0)
    both_dips = np.column_stack((low_dip, up_dip))
    
    # The dip statistic is the maximum difference over all data points
    dip = np.max(both_dips)
    
    return dip

In [ ]:
# Search for CSV files in the directory and its subdirectories
csv_files = glob.glob("../rcv_elections_database/**/*.csv", recursive=True)

# Determine save behavior
save = True

# Dictionary to store all the consistency points across elections
election_consistency_points = {}

# Dictionary to store all the stress values across elections
election_stress_values = {}

# Iterate over the CSV files
for file_path in csv_files:

    # Get the filename only
    filename = file_path.split('/')[-1]  
    print(file_path)

    try:
        csv = file_path
        filename_prefix = "plots/" + filename
            
        # Perform the RCV analysis
        test = perform_rcv_analysis(csv, n_runs=1000)
        mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names, stress = test

        # Print the normalized distances between candidates and plot the MDS analysis
        normalized_distances = get_distances_normalized(most_common_order, mds_1d_coordinates, candidate_names)
        print("Normalized distances:", normalized_distances)
        
        # Check if the plot already exists
        if not os.path.isfile(f"{filename_prefix}_dist.png") and save:
            plot_rcv_distribution(csv_path=file_path, normalized_distances=normalized_distances, save=save, filename=filename_prefix)
        elif save:
            print("Skipping dist plot: {}".format(file_path))
        else:
            plot_rcv_distribution(csv_path=file_path, normalized_distances=normalized_distances, save=save, filename=filename_prefix)

        # Check if the plot already exists
        if not os.path.isfile(f"{filename_prefix}_mds.png") and save:
            plot_rcv_analysis(mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names, save=save, filename=filename_prefix)
        elif save:
            print("Skipping mds plot: {}".format(file_path))
        else:
            plot_rcv_analysis(mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names, save=save, filename=filename_prefix)
        
        # Get the consistency points for the bimodality analysis
        points = get_consistency_points(csv)
        print("Consistency points:", points)
        
        # Store the consistency points for later normalization and plotting
        election_consistency_points[filename] = points

        # Store the stress value for later plotting
        election_stress_values[filename] = stress

        # Create a list of data points
        data_points = []
        for key, value in points.items():
            data_points.extend([key] * value)

        # Convert to numpy array
        data_points = np.array(data_points)

        # Calculate skewness and kurtosis
        g = skew(data_points)
        k = kurtosis(data_points)

        # Calculate KDE without plotting
        density = gaussian_kde(data_points)
        x_vals = np.linspace(min(data_points), max(data_points), 1000)
        y_vals = density(x_vals)

        # Identify local maxima in the KDE
        maxima_indices = argrelextrema(y_vals, np.greater)

        # Get the x-values of the maxima
        modes = x_vals[maxima_indices]

        # Check if there are at least two modes
        if len(modes) >= 2:
            mode1, mode2 = modes[:2]

            # Calculate the amplitudes of the two modes
            amp1 = density(mode1)[0]
            amp2 = density(mode2)[0]

            # Calculate the amplitude ratio with higher amplitude as denominator
            if amp1 > amp2:
                amplitude_ratio = amp2 / amp1
            else:
                amplitude_ratio = amp1 / amp2

            # Calculate x-axis distance between the two modes
            mode_distance = abs(mode2 - mode1)

        else:
            mode1, mode2 = None, None
            amplitude_ratio = None
            mode_distance = None

        try:
            # Split the data into two groups based on proximity to the modes
            data_group1 = [point for point in data_points if abs(point - mode1) < abs(point - mode2)]
            data_group2 = [point for point in data_points if abs(point - mode1) > abs(point - mode2)]

            # Display mean and variance for each group
            mu1, sigma1_sq = np.mean(data_group1), np.var(data_group1)
            mu2, sigma2_sq = np.mean(data_group2), np.var(data_group2)

            # Calculate Ashman's D statistic (D > 2)
            ashmans_D = abs(mu1 - mu2) / np.sqrt((sigma1_sq + sigma2_sq) / 2)

        except:
            mode_distance = 404
            amplitude_ratio = 404
            ashmans_D = 404

        # Calculate Sarle's bimodality coefficient b (b > 5/9)
        n = len(data_points)
        sarle = (g**2 + 1) / (k + 3 * (n-1)**2 / ((n-2) * (n-3)))

        # Calculate Hartigan's dip statistic (calculate p value)
        hartigan_dip = dip_statistic(data_points)

        # Prepare data for histogram
        data_list = [x for x, count in points.items() for _ in range(count)]
        normalized_points = []
        normalized_names = []
        for name in normalized_distances:
            normalized_names.append(name)
            normalized_points.append(normalized_distances[name])

        # Plot histogram
        plt.figure(figsize=(10, 6))
        plt.hist(data_list, bins=50, density=True, alpha=0.7)
        plt.title('Histogram of Data')
        plt.xticks(normalized_points, normalized_names, rotation=45)
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.grid(True)

        # Display measures in a text box
        text_str = f"Skewness: {g:.2f}\nKurtosis: {k:.2f}\nMode Distance: {mode_distance:.2f}"
        plt.text(0.05, 0.95, text_str, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

        # Check if the plot already exists
        if not os.path.isfile(f"{filename_prefix}_hist.png") and save:
            plt.savefig(f"{filename_prefix}_hist.png", bbox_inches='tight')
            plt.close()
        elif save:
            print("Skipping hist plot: {}".format(file_path))
        else:
            plt.show()

        # Plot kernal density estimation
        plt.figure(figsize=(10, 6))
        sns.kdeplot(data_list, fill=True)
        plt.title('Kernel Density Estimation of Data')
        plt.xticks(normalized_points, normalized_names, rotation=45)
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.grid(True)

        # Display bimodality measures in a text box
        text_str = f"Ashman's D: {ashmans_D:.10f}\nSarle's Coefficient: {sarle:.10f}\nHartigan's Dip: {hartigan_dip:.10f}\nAmplitude Ratio: {amplitude_ratio:.10f}"
        plt.text(0.05, 0.95, text_str, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

        # Check if the plot already exists
        if not os.path.isfile(f"{filename_prefix}_kde.png") and save:
            plt.savefig(f"{filename_prefix}_kde.png", bbox_inches='tight')
            plt.close()
        elif save:
            print("Skipping kde plot: {}".format(file_path))
        else:
            plt.show()

    except Exception as e:
        print("Error in file: {}".format(file_path))
        print(e)

In [6]:
# Search for CSV files in the directory and its subdirectories
csv_files = glob.glob("../rcv_elections_database/**/*.csv", recursive=True)

# Dictionary to store all the consistency points across elections
election_consistency_points = {}

# Dictionary to store the candidate distances across elections
election_normalized_distances = {}

# Iterate over the CSV files
for file_path in csv_files:

    # Get the filename only
    filename = file_path.split('/')[-1]  
    print(file_path)

    try:
        csv = file_path
            
        # Perform the RCV analysis
        test = perform_rcv_analysis(csv, n_runs=1000)
        mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names, stress = test

        # Print the normalized distances between candidates and plot the MDS analysis
        normalized_distances = get_distances_normalized(most_common_order, mds_1d_coordinates, candidate_names)
        print("Normalized distances:", normalized_distances)
        
        # Get the consistency points for the bimodality analysis
        points = get_consistency_points(csv)
        print("Consistency points:", points)
        
        # Store the consistency points for later normalization and plotting
        election_consistency_points[filename] = points

        # Store the candidate distances for later analysis
        election_normalized_distances[filename] = normalized_distances

    except Exception as e:
        print("Error in file: {}".format(file_path))
        print(e)

../rcv_elections_database\CandidateDetails.csv
Error in file: ../rcv_elections_database\CandidateDetails.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database\MatchedElections.csv
Error in file: ../rcv_elections_database\MatchedElections.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database\ProportionalRCV.csv
Error in file: ../rcv_elections_database\ProportionalRCV.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database\SequentialRCV.csv
Error in file: ../rcv_elections_database\SequentialRCV.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database\SingleWinnerRCV.csv
Error in file: ../rcv_elections_database\SingleWinnerRCV.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database\classic\Alaska_04102020_PRESIDENTOFTHEUNITEDSTATES.csv


c:\Users\Eric Simpson\Repository\bugs-in-democracy\rcv_exploration\..\rcv_learning\rcv_dimensionality.py:272: RuntimeWarning: invalid value encountered in true_divide
  frequencies = counts / mentioned_together
c:\Users\Eric Simpson\Repository\bugs-in-democracy\rcv_exploration\..\rcv_learning\rcv_dimensionality.py:283: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Normalized distances: {'Tom Steyer': 0.0, 'Amy Klobuchar': 2.2929481554402105, 'Michael R. Bloomberg': 2.4564822253940477, 'Pete Buttigieg': 2.956130820440215, 'Bernie Sanders': 3.4350504601573033, 'Joseph R. Biden': 3.52399068249201, 'Elizabeth Warren': 3.916169474788022, 'Tulsi Gabbard': 7.0}


c:\Users\Eric Simpson\Repository\bugs-in-democracy\rcv_exploration\..\rcv_learning\rcv_dimensionality.py:283: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Consistency points: {3.523990682492009: 2283, 3.457285515740979: 1831, 3.622035380566012: 141, 3.5096039812470377: 19, 3.4350504601573024: 1579, 3.5553302138149823: 435, 3.56542101540793: 15, 3.434926470733162: 30, 3.4556509290144124: 114, 3.495465258850346: 60, 3.635376771965514: 8, 3.468155064684537: 80, 3.2692581660103324: 5, 3.454016342287846: 13, 3.4125674257253453: 34, 4.326287845117977: 151, 3.437949351389531: 14, 3.46076136870047: 19, 3.4763753872956626: 52, 3.098095785953163: 16, 3.4885968234287668: 19, 3.0151979528281627: 3, 2.7233593396685376: 9, 2.5542374789651863: 2, 4.392993011869007: 16, 1.3154988353115011: 1, 3.916169474788021: 27, 3.5510528978095377: 53, 3.5147144209330947: 5, 2.29294815544021: 4, 2.6497311362401614: 3, 3.445147350105277: 15, 0.869880142831164: 3, 3.0356397115723923: 2, 3.3917998021683538: 38, 3.0752003687249005: 2, 7.0: 20, 3.0869782581613245: 9, 3.397420560776343: 32, 0.7720702880042578: 1, 2.701124284084861: 5, 3.433009184469575: 9, 3.51982486061915

...

Consistency points: {0.8229892886172783: 26, 2.0: 56, 0.3528736610771598: 122, 0.0: 46, 0.20574732215431957: 53, 1.014368305385799: 52, 1.1172419664629587: 3, 0.5: 10}
../rcv_elections_database\single\Wyoming_04172020_PRESIDENTOFTHEUNITEDSTATES.csv


c:\Users\Eric Simpson\Repository\bugs-in-democracy\rcv_exploration\..\rcv_learning\rcv_dimensionality.py:283: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Normalized distances: {'Tulsi Gabbard': 0.0, '(undeclared)': 1.942480002111236, 'Bernie Sanders': 3.739536723719921, 'Joseph R. Biden': 4.053744908949053, 'Elizabeth Warren': 4.405547806343052, 'Amy Klobuchar': 4.812944095905922, 'Michael R. Bloomberg': 5.1640378261808495, 'Pete Buttigieg': 6.3504836891363645, 'Tom Steyer': 8.0}


c:\Users\Eric Simpson\Repository\bugs-in-democracy\rcv_exploration\..\rcv_learning\rcv_dimensionality.py:283: RuntimeWarning: divide by zero encountered in true_divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Consistency points: {3.9321740703223806: 636, 3.8788741893405407: 2402, 3.8611075623465942: 3270, 4.439341848234212: 12, 4.099066695736426: 182, 1.9424800021112356: 107, 0.0: 32, 4.186840128305158: 81, 4.050517504327549: 154, 3.997217623345709: 136, 0.9741602040836219: 16, 4.781737961779738: 2, 6.3504836891363645: 8, 4.048142159541068: 59, 4.536751475025877: 8, 4.483451594044037: 93, 4.2912591448902315: 84, 3.842848563117473: 9, 6.451690751003552: 2, 4.335145861174597: 23, 4.061207459033492: 1, 4.830524452788558: 1, 4.895830671759946: 21, 4.0920288758254335: 14, 4.50456275131999: 3, 5.1640378261808495: 13, 3.6164282504431022: 3, 4.320201047884489: 3, 3.660844817927969: 2, 4.273694302995878: 1, 3.8018057458011554: 9, 4.405547806343052: 7, 0.9830435175805952: 12, 0.9652768905866486: 10, 4.16046183188245: 1, 4.388445742156438: 1, 4.5957109822252935: 1, 4.1557021087890895: 1, 4.196253293002632: 1, 4.560900340908795: 6, 3.8592382507070067: 6, 1.5876209222840911: 2, 3.7042016830118336: 2, 4.

In [7]:
import pickle

# Specify the filename for the pickle file
pickle_filename = "election_normalized_distances.pkl"

# Open the file in write-binary mode and dump the dictionary
with open(pickle_filename, 'wb') as f:
    pickle.dump(election_normalized_distances, f)

In [ ]:
import pickle

# Specify the filename for the pickle file
pickle_filename = "election_stress_values.pkl"

# Open the pickle file in read-binary mode and load the dictionary
with open(pickle_filename, 'rb') as f:
    election_stress_values = pickle.load(f)

for key, value in election_stress_values.items():
    print(key, value)

In [ ]:
import pickle

# Specify the filename for the pickle file
pickle_filename = "election_consistency_points.pkl"

# Open the file in write-binary mode and dump the dictionary
with open(pickle_filename, 'wb') as f:
    pickle.dump(election_consistency_points, f)

In [ ]:
import pickle

# Specify the filename for the pickle file
pickle_filename = "election_consistency_points.pkl"

# Open the pickle file in read-binary mode and load the dictionary
with open(pickle_filename, 'rb') as f:
    election_consistency_points = pickle.load(f)

for key, value in election_consistency_points.items():
    print(key, value)

In [ ]:
import pandas as pd

def filter_elections(filter_criteria):
    """Filter the elections based on the given criteria."""
    election_df = pd.read_csv("election_table.csv")
    
    # Apply the filters
    for key, value in filter_criteria.items():
        if isinstance(value, list):  # Check if the filter value is a list
            election_df = election_df[election_df[key].isin(value)]  # Use .isin() for lists
        else:
            election_df = election_df[election_df[key] == value]
    
    filtered_elections = election_df['filename'].tolist()
    election_list = [election_consistency_points[election] for election in filtered_elections if election in election_consistency_points]
    for election in filtered_elections:
        print(election)
    
    return election_list

In [ ]:
from scipy.stats import gaussian_kde
import numpy as np

def compute_kdes(points_list):
    """Compute KDE for each election."""
    
    kdes = []
    x_grid = np.linspace(0, 1, 500)  # Define a grid to evaluate the KDEs
    
    # Normalize and compute KDE for each election
    for points in points_list:
        # Normalize x-values
        abs_max_x = max(abs(min(points.keys())), abs(max(points.keys())))
        normalized_data = [(x + abs_max_x) / (2 * abs_max_x) for x in points.keys()]
        
        # Compute KDE and evaluate it on the x_grid
        kde = gaussian_kde(normalized_data)
        kde_values = kde(x_grid)
        
        kdes.append(kde_values)
    
    return kdes, x_grid

In [ ]:
import matplotlib.pyplot as plt

def plot_kdes(kdes, x_grid, title="KDE Analysis"):
    """Plot the computed KDEs."""
    
    plt.figure(figsize=(10, 6))
    
    for kde_values in kdes:
        plt.plot(x_grid, kde_values)
    
    plt.title(title)
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

In [ ]:
# Filter and compute KDE for STATE and FEDERAL level elections with YES in partisan
filter_criteria = {'level': ['FEDERAL', 'STATE'], 'partisan': 'YES'}
state_federal_points = filter_elections(filter_criteria)
print("Computing KDE for STATE and FEDERAL level elections with YES in partisan:")
kdes, x_grid = compute_kdes(state_federal_points)
plot_kdes(kdes, x_grid, "Partisan State and Federal Elections KDEs")

# Filter and compute KDE for LOCAL elections with NO in partisan
filter_criteria = {'level': 'LOCAL', 'partisan': 'NO'}
local_points = filter_elections(filter_criteria)
print("Computing KDE for LOCAL elections with NO in partisan:")
kdes, x_grid = compute_kdes(local_points)
plot_kdes(kdes, x_grid, "Non-Partisan Local Elections KDEs")

# Compute and plot KDE for a single election
print("Computing KDE for target election:")
kdes, x_grid = compute_kdes([election_consistency_points["Alaska_11082022_USRepresentative.csv"]])
plot_kdes(kdes, x_grid, "Alaska_11082022_USRepresentative.csv KDE")

In [ ]:
def mirror_and_normalize(points):
    # 1. Determine the x-axis midpoint
    midpoint = (max(points.keys()) + min(points.keys())) / 2

    # 2. Create a mirrored version of the dataset
    mirrored_points = {midpoint + (midpoint - k): v for k, v in points.items()}

    # 3. Combine the original and mirrored dataset
    combined = {}
    for k, v in points.items():
        combined[k] = v + mirrored_points.get(k, 0)
    for k, v in mirrored_points.items():
        combined[k] = v + points.get(k, 0)

    # 4. Normalize the x-values
    min_x = min(combined.keys())
    max_x = max(combined.keys())
    x_range = max_x - min_x
    combined = {(k - min_x) / x_range: v for k, v in combined.items()}

    # 5. Normalize the y-values
    total_y = sum(combined.values())
    normalized = {k: v / total_y for k, v in combined.items()}

    return normalized

def average_datasets(points_list):
    normalized_list = [mirror_and_normalize(points) for points in points_list]

    # A dictionary to store the sum of y-values for each x-value
    y_sum = {}
    # A dictionary to store the count of y-values for each x-value
    y_count = {}

    for normalized_points in normalized_list:
        for k, v in normalized_points.items():
            if k in y_sum:
                y_sum[k] += v
                y_count[k] += 1
            else:
                y_sum[k] = v
                y_count[k] = 1

    # Compute the average y-value for each x-value
    averaged = {k: y_sum[k] / y_count[k] for k in y_sum.keys()}
    
    # Normalize the peak magnitude to 1
    max_y_value = max(averaged.values())
    normalized_averaged = {k: v / max_y_value for k, v in averaged.items()}

    return normalized_averaged

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def average_and_plot(points_list, title="Averaged Data Analysis"):
    
    # Use the mirror_and_normalize function to handle the mirroring and normalization
    normalized_points_list = [mirror_and_normalize(points) for points in points_list]

    # Average across the normalized dictionaries
    averaged_points = {}
    for normalized_points in normalized_points_list:
        for k, v in normalized_points.items():
            if k in averaged_points:
                averaged_points[k] += v
            else:
                averaged_points[k] = v

    num_dicts = len(normalized_points_list)
    averaged_points = {k: v / num_dicts for k, v in averaged_points.items()}

    # Extract data for histogram and KDE
    data_list = [x for x, count in averaged_points.items() for _ in range(int(count * 100000))]  # Scale the values for visualization

    # Plot histogram
    plt.figure(figsize=(10, 6))
    plt.hist(data_list, bins=50, density=True, alpha=0.7)
    plt.title(f"{title} - Histogram")
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

    # Plot kernal density estimation
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data_list, fill=True)
    plt.title(f"{title} - Kernel Density Estimation")
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def average_and_plot(points_list, title="Averaged Data Analysis"):
    
    # Normalize and scale the dictionaries individually
    normalized_points_list = []
    for points in points_list:
        
        # Determine the absolute max x-value for symmetric normalization for this set of points
        abs_max_x = max(abs(min(points.keys())), abs(max(points.keys())))
        
        # Symmetrically normalize x-values
        normalized_x = {x: (x + abs_max_x) / (2 * abs_max_x) for x in points.keys()}
        
        # Find the peak y-value (highest frequency)
        max_y = max(points.values())
        
        # Scale y-values so the peak is 1
        scaled_y = {k: v / max_y for k, v in points.items()}
        
        # Create normalized dataset
        normalized_points = {normalized_x[k]: scaled_y[k] for k in points.keys()}
        normalized_points_list.append(normalized_points)

    # Average across the normalized dictionaries
    averaged_points = {}
    counts = {}  # To store counts for averaging

    for normalized_points in normalized_points_list:
        for k, v in normalized_points.items():
            if k in averaged_points:
                averaged_points[k] += v
                counts[k] += 1
            else:
                averaged_points[k] = v
                counts[k] = 1

    # Now divide the summed y-values by their counts to get the average
    averaged_points = {k: averaged_points[k] / counts[k] for k in averaged_points.keys()}

    # Extract data for histogram and KDE
    data_list = [x for x, count in averaged_points.items() for _ in range(int(count * 100000))]  # Scale the values for visualization

    # Plot histogram
    plt.figure(figsize=(10, 6))
    plt.hist(data_list, bins=50, density=True, alpha=0.7)
    plt.title(f"{title} - Histogram")
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

    # Plot kernal density estimation
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data_list, fill=True)
    plt.title(f"{title} - Kernel Density Estimation")
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

In [ ]:
# Filter and average for STATE and FEDERAL level elections with YES in partisan
filter_criteria = {'level': ['FEDERAL', 'STATE'], 'partisan': 'YES'}
state_federal_points = filter_elections(filter_criteria)
print("(Method 1) Averaging for STATE and FEDERAL level elections with YES in partisan:")
average_and_plot(state_federal_points, "Partisan State and Federal Elections")

# Filter and average for LOCAL elections with NO in partisan
filter_criteria = {'level': 'LOCAL', 'partisan': 'NO'}
local_points = filter_elections(filter_criteria)
print("(Method 1) Averaging for LOCAL elections with NO in partisan:")
average_and_plot(local_points, "Non-Partisan Local Elections")

# Testing for a single election
print("Testing for target election:")
average_and_plot([election_consistency_points["Alaska_11082022_USRepresentative.csv"]], "Alaska_11082022_USRepresentative.csv")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
import numpy as np

def average_and_plot(points_list, title="Averaged Data Analysis"):
    
    # Grid for KDE evaluation
    x_grid = np.linspace(0, 1, 500)
    kde_values_list = []
    
    # Normalize, scale and compute KDE for each election
    for points in points_list:
        # Normalize x-values
        abs_max_x = max(abs(min(points.keys())), abs(max(points.keys())))
        normalized_data = [(x + abs_max_x) / (2 * abs_max_x) for x in points.keys()]
        
        # Scale y-values
        max_y = max(points.values())
        scaled_weights = [v / max_y for v in points.values()]
        
        # Compute KDE values
        kde = gaussian_kde(normalized_data, weights=scaled_weights)
        kde_values_list.append(kde(x_grid))
    
    # Average the KDE values across all elections
    avg_kde = np.mean(kde_values_list, axis=0)
    
    # Plot the averaged KDE
    plt.figure(figsize=(10, 6))
    plt.plot(x_grid, avg_kde)
    plt.title(f"{title} - Kernel Density Estimation")
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

In [ ]:
# Filter and average for STATE and FEDERAL level elections with YES in partisan
filter_criteria = {'level': ['FEDERAL', 'STATE'], 'partisan': 'YES'}
state_federal_points = filter_elections(filter_criteria)
print("(Method 1) Averaging for STATE and FEDERAL level elections with YES in partisan:")
average_and_plot(state_federal_points, "Partisan State and Federal Elections")

# Filter and average for LOCAL elections with NO in partisan
filter_criteria = {'level': 'LOCAL', 'partisan': 'NO'}
local_points = filter_elections(filter_criteria)
print("(Method 1) Averaging for LOCAL elections with NO in partisan:")
average_and_plot(local_points, "Non-Partisan Local Elections")

# Testing for a single election
print("Testing for target election:")
average_and_plot([election_consistency_points["Alaska_11082022_USRepresentative.csv"]], "Alaska_11082022_USRepresentative.csv")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import euclidean
from scipy.stats import entropy

def filter_elections(filter_criteria, election_data_dict):
    """Filter the elections based on the given criteria."""
    election_df = pd.read_csv("election_table.csv")
    
    # Apply the filters
    for key, value in filter_criteria.items():
        if isinstance(value, list):
            election_df = election_df[election_df[key].isin(value)]
        else:
            election_df = election_df[election_df[key] == value]
    
    filtered_elections = election_df['filename'].tolist()
    filtered_data_dict = {election: election_data_dict[election] for election in filtered_elections if election in election_data_dict}
    
    return filtered_data_dict

def calculate_distance_matrix(election_data, distance_func):
    num_samples = len(election_data)
    distance_matrix = np.zeros((num_samples, num_samples))
    
    election_keys = list(election_data.keys())
    for i in range(num_samples):
        for j in range(i+1, num_samples):
            x1, y1 = election_data[election_keys[i]]
            x2, y2 = election_data[election_keys[j]]
            
            distance = distance_func(y1, y2)
            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance
            
    return distance_matrix

def kullback_leibler(p, q):
    return entropy(p, q)

def hierarchical_clustering_and_dendrogram(distance_matrix, labels, title, orientation='right'):
    linkage_matrix = linkage(distance_matrix, 'average', optimal_ordering=True)
    plt.figure(figsize=(16, 9))
    dendrogram(linkage_matrix, labels=labels, orientation=orientation)
    plt.title(title)
    plt.show()

# Example usage
# This assumes you have a dictionary named 'election_data_dict' where the keys are filenames and the values are (x, y) tuples of the KDEs.

# Filtering based on criteria
filter_criteria = {'level': ['FEDERAL', 'STATE'], 'partisan': 'YES'}
filtered_election_data = filter_elections(filter_criteria, election_kde_data)  # Assuming election_kde_data is your complete data

# Calculating distance matrix using Euclidean distance
distance_matrix = calculate_distance_matrix(filtered_election_data, euclidean)

# Performing hierarchical clustering and plotting dendrogram
labels = list(filtered_election_data.keys())
hierarchical_clustering_and_dendrogram(distance_matrix, labels, 'Filtered - Euclidean Distance')